In [3]:
import ehr_utils, ehr_models
import os
import pickle
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils import compute_sample_weight
# from imblearn.over_sampling import SMOTE
import shutil


In [ ]:
print("=== 步骤1：获取训练集和测试集 ===")
X_train, X_test, y_train, y_test = ehr_utils.preprocess_ehr_train_test_data('data_processed/benbu_baseline_cleaned_onehot_simulated80.csv')
print(f"原始训练集大小：{X_train.shape}")
print(f"原始测试集大小：{X_test.shape}")

print("\n=== 步骤2：准备所有模型 ===")
# 定义所有要训练的模型
all_models = [
    'AdaBoost', 'DecisionTree', 'GaussianNB', 'GradientBoosting', 'LightGBM', 'LinearDiscriminantAnalysis', 'LogisticRegression', 'MLPClassifier', 'RandomForest', 'XGBoost'
]
print(f"将训练以下 {len(all_models)} 个模型：")
for i, model_name in enumerate(all_models, 1):
    print(f"  {i}. {model_name}")

# 存储所有模型的结果
all_models_results = {}
total_start_time = time.time()
print("\n" + "=" * 80)
print("开始训练所有模型...")
print("-" * 80)
shutil.rmtree('models', ignore_errors=True)
os.makedirs('models', exist_ok=True)

n_splits = 5

for model_idx, model_name in enumerate(all_models, 1):
    if model_idx<7:
        continue
    print(f"\n{'='*60}")
    print(f"正在训练模型 {model_idx}/{len(all_models)}：{model_name}")
    print(f"{'='*60}")
    model_start_time = time.time()

    try:
        # 获取模型
        model = ehr_models.get_model(model_name)
        print(f"模型参数：{model.get_params()}")
        print(f"\n=== 开始 {model_name} 五折交叉验证 ===")
        # 交叉验证设置
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        cv_results = {'train_aucs': [], 'val_aucs': [], 'models': [], 'fold_results': []}

        # K折交叉验证
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
            print(f'\n----- Fold {fold + 1}/{n_splits} -----')
            X_fold_train = X_train.iloc[train_idx]
            X_fold_val = X_train.iloc[val_idx]
            y_fold_train = y_train.iloc[train_idx].values  
            y_fold_val = y_train.iloc[val_idx].values  

            # 计算样本权重
            sample_weights = compute_sample_weight(class_weight='balanced', y=y_fold_train)
            pos_mask = (y_fold_train == 1)
            sample_weights[pos_mask] *= 1.1

            # 每次都重新获取模型
            fold_model = ehr_models.get_model(model_name)

            if model_name == 'LinearDiscriminantAnalysis':
                # LDA 不支持样本权重，使用 SMOTE
                smote = SMOTE(random_state=42)
                X_train_fold_res, y_train_fold_res = smote.fit_resample(X_fold_train, y_fold_train)  # type: ignore
                fold_model.fit(X_train_fold_res, y_train_fold_res)  # type: ignore
            else:
                fold_model.fit(X_fold_train, y_fold_train, sample_weight=sample_weights)  # type: ignore

            # 预测
            y_train_proba = np.asarray(fold_model.predict_proba(X_fold_train))[:, 1]
            y_val_proba = np.asarray(fold_model.predict_proba(X_fold_val))[:, 1]

            # 计算 AUC
            train_auc = roc_auc_score(y_fold_train, y_train_proba)
            val_auc = roc_auc_score(y_fold_val, y_val_proba)
            print(f"训练集 AUC：{train_auc:.4f}，验证集 AUC：{val_auc:.4f}，过拟合：{train_auc - val_auc:.4f}")
            # 保存结果
            cv_results['train_aucs'].append(train_auc)
            cv_results['val_aucs'].append(val_auc)
            cv_results['models'].append(fold_model)
            cv_results['fold_results'].append({'fold': fold, 'train_auc': train_auc, 'val_auc': val_auc, 'train_idx': train_idx, 'val_idx': val_idx})

        # 计算平均结果
        mean_train_auc = np.mean(cv_results['train_aucs'])
        mean_val_auc = np.mean(cv_results['val_aucs'])
        std_train_auc = np.std(cv_results['train_aucs'])
        std_val_auc = np.std(cv_results['val_aucs'])

        print(f"\n{model_name} 五折交叉验证结果汇总：")
        print(f"各折验证集 AUC: {[f'{auc:.4f}' for auc in cv_results['val_aucs']]}")
        print(f"平均训练集 AUC: {mean_train_auc:.4f} ± {std_train_auc:.4f}")
        print(f"平均验证集 AUC: {mean_val_auc:.4f} ± {std_val_auc:.4f}")
        print(f"平均过拟合程度：{mean_train_auc - mean_val_auc:.4f}")
        # 选择最佳模型（验证集 AUC 最高的）
        best_fold_idx = np.argmax(cv_results['val_aucs'])
        best_model = cv_results['models'][best_fold_idx]
        best_val_auc = cv_results['val_aucs'][best_fold_idx]
        print(f"最佳模型来自第 {best_fold_idx+1} 折，验证集 AUC：{best_val_auc:.4f}")

        # 保存最佳模型
        os.makedirs('models', exist_ok=True)
        model_filename = f'models/{model_name}_cv_{mean_val_auc:.4f}.pkl'
        with open(model_filename, 'wb') as f:
            pickle.dump(best_model, f)
        print(f"最佳模型已保存到：{model_filename}")

        # 计算训练时间
        model_end_time = time.time()
        model_duration = model_end_time - model_start_time

        # 保存模型结果
        all_models_results[model_name] = {
            'mean_train_auc': mean_train_auc,
            'mean_val_auc': mean_val_auc,
            'std_train_auc': std_train_auc,
            'std_val_auc': std_val_auc,
            'best_val_auc': best_val_auc,
            'overfitting': mean_train_auc - mean_val_auc,
            'training_time': model_duration,
            'best_model': best_model,
            'cv_results': cv_results
        }
        print(f"✓ {model_name} 训练完成！用时：{model_duration:.2f} 秒")

    except Exception as e:
        print(f"✗ {model_name} 训练失败：{str(e)}")
        all_models_results[model_name] = {'error': str(e), 'training_time': time.time() - model_start_time}

# 计算总用时
total_end_time = time.time()
total_duration = total_end_time - total_start_time

print("\n" + "=" * 80)
print("所有模型训练完成！")
print("=" * 80 + "\n\n")

=== 步骤1：获取训练集和测试集 ===
总特征数：96，需要log1p变换的特征数：66
训练集样本数：8000，验证集样本数：2000
原始训练集大小：(8000, 96)
原始测试集大小：(2000, 96)

=== 步骤2：准备所有模型 ===
将训练以下 10 个模型：
  1. AdaBoost
  2. DecisionTree
  3. GaussianNB
  4. GradientBoosting
  5. LightGBM
  6. LinearDiscriminantAnalysis
  7. LogisticRegression
  8. MLPClassifier
  9. RandomForest
  10. XGBoost

开始训练所有模型...
--------------------------------------------------------------------------------

正在训练模型 7/10：LogisticRegression
模型参数：{'C': 9.975836024182009, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'deprecated', 'n_jobs': -1, 'penalty': 'l2', 'random_state': 42, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

=== 开始 LogisticRegression 五折交叉验证 ===

----- Fold 1/5 -----


c:\Users\mclab\.conda\envs\ehr\Lib\site-packages\sklearn\linear_model\_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


训练集 AUC：0.9957，验证集 AUC：0.9910，过拟合：0.0048

----- Fold 2/5 -----


c:\Users\mclab\.conda\envs\ehr\Lib\site-packages\sklearn\linear_model\_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


训练集 AUC：0.9956，验证集 AUC：0.9937，过拟合：0.0020

----- Fold 3/5 -----


...

训练集 AUC：0.9825，验证集 AUC：0.9590，过拟合：0.0235

----- Fold 4/5 -----


c:\Users\mclab\.conda\envs\ehr\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:45:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\mclab\.conda\envs\ehr\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:45:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)


训练集 AUC：0.9833，验证集 AUC：0.9631，过拟合：0.0202

----- Fold 5/5 -----


c:\Users\mclab\.conda\envs\ehr\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:45:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\mclab\.conda\envs\ehr\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:45:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)


训练集 AUC：0.9826，验证集 AUC：0.9533，过拟合：0.0293

XGBoost 五折交叉验证结果汇总：
各折验证集 AUC: ['0.9441', '0.9702', '0.9590', '0.9631', '0.9533']
平均训练集 AUC: 0.9827 ± 0.0005
平均验证集 AUC: 0.9580 ± 0.0088
平均过拟合程度：0.0247
最佳模型来自第 2 折，验证集 AUC：0.9702
最佳模型已保存到：models/XGBoost_cv_0.9580.pkl
✓ XGBoost 训练完成！用时：11.25 秒

所有模型训练完成！
